In [6]:
# criar um navegador
import re
import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# importar/visualizar a base de dados

df_buscas = pd.read_excel('buscas.xlsx')   
display(df_buscas)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 14 128 gb,mini watch vitrine usado,3000,5500
1,rtx 3060,zota galax,1700,3500


In [10]:
def find_google(driver, product, banned_terms, min_price, max_price):
    list_google = []

    banned_terms = banned_terms.lower()
    list_banned_terms = banned_terms.split(' ')
    list_product_terms = product.split(' ')

    # Pesquisar o nome do produto no Google
    driver.get('https://www.google.com.br')
    sleep(1)
    # Localizar a caixa de pesquisa e enviar o nome do produto
    driver.find_element(By.CLASS_NAME, 'gLFyf').send_keys(product, Keys.ENTER)

    sleep(1)
    list_items = driver.find_elements(By.XPATH, '//div[@role="listitem"]')

    sleep(1)
    for item in list_items:
        if 'Shopping' in item.text:
            item.click()
            break
    sleep(1)
    # Pegar a lista de resultados da busca no Google Shopping
    list_results = WebDriverWait(driver, 2).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'i0X6df')))

    # Processa os resultados
    for result in list_results:
        try:
            # Pega o nome do produto
            name = WebDriverWait(result, 2).until(EC.presence_of_element_located((By.TAG_NAME, 'h3'))).text.lower()

            # Verifica se o nome contém termos banidos
            banned_terms_found = False
            for banned_term in list_banned_terms:
                if banned_term in name:
                    banned_terms_found = True
                    break

            # Verifica se o nome contém todos os termos do produto
            product_terms_found = True
            for product_term in list_product_terms:
                if product_term not in name:
                    product_terms_found = False
                    break

            # Se o nome não contém termos banidos e contém os termos do produto
            if not banned_terms_found and product_terms_found:
                # Pega o preço do produto
                price_element = WebDriverWait(result, 2).until(EC.presence_of_element_located((By.CLASS_NAME, 'a8Pemb')))
                price_text = price_element.text
                price = float(re.sub('[^0-9,.]', '', price_text).replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.'))
                

                # Verifica se o preço está dentro da faixa desejada
                if min_price <= price <= max_price:
                    # Pega o link do produto
                    element_ref = result.find_element(By.CLASS_NAME, 'bONr3b')
                    elemento_pai = element_ref.find_element(By.XPATH, '..')  
                    link = elemento_pai.get_attribute('href')
                    
                    list_google.append((name, price, link))

        except Exception as e:
            print(f"Erro ao processar item: {e}")
            continue

    return list_google


def find_buscape(driver, product, banned_terms, min_price, max_price):
    list_buscape = []

    banned_terms = banned_terms.lower()
    list_banned_terms = banned_terms.split(' ')
    list_product_terms = product.split(' ')

    # Pesquisar o nome do produto no Google
    driver.get('https://www.buscape.com.br/')
    sleep(1)
    # Localizar a caixa de pesquisa e enviar o nome do produto
    driver.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(product, Keys.ENTER)

    sleep(1)
    # Pegar a lista de resultados da busca no Google Shopping
    list_results = WebDriverWait(driver, 2).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'Hits_ProductCard__Bonl_')))

    # Processa os resultados
    for result in list_results:
        try:
            # Pega o nome do produto
            name = WebDriverWait(result, 2).until(EC.presence_of_element_located((By.XPATH, '//*[@data-testid="product-card::name"]'))).text.lower()

            # Verifica se o nome contém termos banidos
            banned_terms_found = False
            for banned_term in list_banned_terms:
                if banned_term in name:
                    banned_terms_found = True
                    break

            # Verifica se o nome contém todos os termos do produto
            product_terms_found = True
            for product_term in list_product_terms:
                if product_term not in name:
                    product_terms_found = False
                    break

            # Se o nome não contém termos banidos e contém os termos do produto
            if not banned_terms_found and product_terms_found:
                # Pega o preço do produto
                price_element = WebDriverWait(result, 2).until(EC.presence_of_element_located((By.XPATH, '//*[@data-testid="product-card::price"]')))
                price_text = price_element.text
                price = float(re.sub('[^0-9,.]', '', price_text).replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.'))
                            
            
                # Verifica se o preço está dentro da faixa desejada
                
                if min_price <= price <= max_price:
                    # Pega o link do produto
                    link  = result.find_element(By.XPATH, '//a[@data-testid="product-card::card"]').get_attribute('href')
                    #elemento_pai = element_ref.find_element(By.XPATH, '..')  
                    #link = elemento_pai.get_attribute('href')
                    
                    list_buscape.append((name, price, link))    

        except Exception as e:
            print(f"Erro ao processar item: {e}")
            continue

    return list_buscape


In [ ]:
# Criar o DataFrame para armazenar os resultados
df_complete = pd.DataFrame()

# Criar o driver do Selenium
driver = webdriver.Chrome()

# Loop para processar cada item do df_buscas
for item in df_buscas.index:
    # Obtendo os dados para a busca
    product = df_buscas.loc[item, 'Nome']
    banned_terms = df_buscas.loc[item, 'Termos banidos']
    min_price = df_buscas.loc[item, 'Preço mínimo']
    max_price = df_buscas.loc[item, 'Preço máximo']

    print(f"\nProcessando item {item + 1}: {product}")  # Debug: Verificar qual item está sendo processado

    # Buscar resultados no Google
    list_google = find_google(driver, product, banned_terms, min_price, max_price)
    
    if list_google:
        # Criar DataFrame a partir dos resultados do Google
        df_google = pd.DataFrame(list_google, columns=['Nome', 'Preço', 'Link'])
        df_complete = pd.concat([df_complete, df_google], ignore_index=True)
    else:
        print("Nenhum resultado do Google encontrado.")  # Debug
    
    
    # Buscar resultados no Buscapé
    list_buscape = find_buscape(driver, product, banned_terms, min_price, max_price)
    
    if list_buscape:
        # Criar DataFrame a partir dos resultados do Buscapé
        df_buscape = pd.DataFrame(list_buscape, columns=['Nome', 'Preço', 'Link'])
        # Concatenar ao DataFrame acumulador
        df_complete = pd.concat([df_complete, df_buscape], ignore_index=True)
    else:
        print("Nenhum resultado do Buscapé encontrado.")  # Debug

# Fechar o driver do Selenium
driver.quit()

# Exibir o DataFrame final
print("Resultados finais combinados:")



Processando item 1: iphone 14 128 gb
Resultados do Google: [('iphone 14 128gb - preto - sou como novo', 3474.55, 'https://www.google.com.br/url?url=https://www.magazineluiza.com.br/usado-iphone-14-preto-128gb-excelente-trocafy-apple/p/ffj3h7a129/te/ip14/%3Fseller_id%3Dtrocafy%26srsltid%3DAfmBOoptG9pRoFrEDNqV7Bza-1UiFx1Nx4hCXU3ZnqPkupyO2iZzu51cr9Y&rct=j&q=&esrc=s&opi=95576897&sa=U&ved=0ahUKEwjX992jtfCKAxXVlZUCHeM7Nc8QguUECLIM&usg=AOvVaw1rH-PBiDSeYdFQTr5NGK4M'), ('apple iphone 14 128gb 6,1” 12mp ios 5g', 3219.0, 'https://www.google.com.br/url?url=https://melhorcompra.space/produto/apple-iphone-14-128gb-61-12mp-ios-5g/%3Fsrsltid%3DAfmBOorEWsrbKZYjs6SRSAMZFZhktCRTwj8b9BERcVXj66Bq_-Rp3jdlulw&rct=j&q=&esrc=s&opi=95576897&sa=U&ved=0ahUKEwjX992jtfCKAxXVlZUCHeM7Nc8QgOUECMMM&usg=AOvVaw2FuU8MjNknrsCe0SV5FZLI'), ('apple iphone 14 plus 128gb meia-noite', 4299.0, 'https://www.google.com.br/url?url=https://www.amazon.com.br/Apple-iPhone-Plus-128-Cinza-Escuro/dp/B07DNQQT62%3Fsource%3Dps-sl-shoppingad

In [13]:
display(df_complete)

,Nome,Preço,Link
0,iphone 14 128gb - preto - sou como novo,3474.55,https://www.google.com.br/url?url=https://www....
1,"apple iphone 14 128gb 6,1” 12mp ios 5g",3219.00,https://www.google.com.br/url?url=https://melh...
2,apple iphone 14 plus 128gb meia-noite,4299.00,https://www.google.com.br/url?url=https://www....
3,"iphone 14 apple (128gb) meia-noite, tela de 6,...",4554.00,https://www.google.com.br/url?url=https://www....
4,apple iphone 14 128gb estelar,5219.10,https://www.google.com.br/url?url=https://www....
...,...,...,...
76,placa de vídeo - geforce rtx 3060 12gb gddr6,2180.00,https://www.google.com.br/url?url=https://www....
77,placa de vídeo inno3d geforce rtx 3060 twin x2...,2240.99,https://www.google.com.br/url?url=https://www....
78,placa de vídeo pny rtx 3060 revel epic-x rgb 1...,1892.84,https://www.google.com.br/url?url=http://www.r...
79,placa de vídeo rtx 3060 8gb pegasus gd6 128bit...,1999.90,https://www.google.com.br/url?url=https://www....


In [14]:
df_complete.to_excel('resultados.xlsx', index=False)